In [1]:
using BenchmarkTools
using LinearAlgebra
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation

LinearAlgebraicRepresentation

In [2]:
square = Lar.cuboid([1,1])

([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0], [[1, 2, 3, 4]])

In [3]:
table = Lar.apply(Lar.t(-0.5,-0.5),square)

([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]])

In [4]:
structure = Lar.Struct([repeat([table,Lar.r(pi/2)],outer=2)...])

LinearAlgebraicRepresentation.Struct(Any[([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]]), [6.123233995736766e-17 -1.0 0.0; 1.0 6.123233995736766e-17 0.0; 0.0 0.0 1.0], ([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]]), [6.123233995736766e-17 -1.0 0.0; 1.0 6.123233995736766e-17 0.0; 0.0 0.0 1.0]], [[-0.5; -0.5], [0.5; 0.5]], "13132074318155819507", 2, "feature")

In [5]:
structure.box

2-element Array{Array{Float64,2},1}:
 [-0.5; -0.5]
 [0.5; 0.5]

## Vecchia funzione

In [6]:
function struct2lar(structure)
	listOfModels = Lar.evalStruct(structure)
	vertDict = Dict()
	index,defaultValue = 0,0
	W = Array{Float64,1}[]
	m = length(listOfModels[1])
	larmodel = [Array{Number,1}[] for k=1:m]

	for model in listOfModels
		V = model[1]
		for k=2:m
			for incell in model[k]
				outcell=[]
				for v in incell
					key = map(Lar.approxVal(7), V[:,v])
					if get(vertDict,key,defaultValue)==defaultValue
						index += 1
						vertDict[key]=index
						push!(outcell,index)
						push!(W,key)
					else
						push!(outcell,vertDict[key])
					end
				end
				append!(larmodel[k],[outcell])
			end
		end
	end

	append!(larmodel[1], W)
	V = hcat(larmodel[1]...)
	chains = [convert(Lar.Cells, chain) for chain in larmodel[2:end]]
	return (V, chains...)
end

struct2lar (generic function with 1 method)

In [7]:
@btime struct2lar(structure)

  33.299 μs (145 allocations: 8.86 KiB)


([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4], [3, 1, 4, 2]])

In [8]:
@code_warntype struct2lar(structure)

Variables
  #self#::Core.Compiler.Const(struct2lar, false)
  structure::LinearAlgebraicRepresentation.Struct
  #3::var"#3#5"
  #4::var"#4#6"
  listOfModels::Any
  vertDict::Dict{Any,Any}
  index::Int64
  defaultValue::Int64
  W::Array{Array{Float64,1},1}
  m::Any
  larmodel::Any
  @_12::Any
  V::Any
  chains::Any
  model::Any
  @_16::Any
  k::Any
  @_18::Any
  incell::Any
  outcell::Array{Any,1}
  @_21::Any
  v::Any
  key::Any

Body::Tuple{Any,Vararg{Any,N} where N}
1 ──        Core.NewvarNode(:(#4))
│           Core.NewvarNode(:(V))
│           Core.NewvarNode(:(chains))
│    %4   = Base.getproperty(Main.Lar, :evalStruct)::Any
│           (listOfModels = (%4)(structure))
│           (vertDict = Main.Dict())
│           (index = 0)
│           (defaultValue = 0)
│    %9   = Core.apply_type(Main.Array, Main.Float64, 1)::Core.Compiler.Const(Array{Float64,1}, false)
│           (W = Base.getindex(%9))
│    %11  = Base.getindex(listOfModels, 1)::Any
│           (m = Main.length(%11))
│    

## Nuove funzioni

In [9]:
function flatten(listOfModels)
	W = Array{Float64,1}[]
	m = length(listOfModels[1])
	larmodel = [Array{Number,1}[] for k=1:m]
	vertDict = Dict()
	index,defaultValue = 0,0
	for model in listOfModels
		V = model[1]
		for k=2:m
			for incell in model[k]
				outcell=[]
				for v in incell
					key = map(Lar.approxVal(7), V[:,v])
					if get(vertDict,key,defaultValue)==defaultValue
						index += 1
						vertDict[key]=index
						push!(outcell,index)
						push!(W,key)
					else
						push!(outcell,vertDict[key])
					end
				end
				append!(larmodel[k],[outcell])
			end
		end
	end
	return larmodel,W
end

flatten (generic function with 1 method)

In [10]:
function struct2lar(structure) 
	larmodel,W = flatten(Lar.evalStruct(structure))
	append!(larmodel[1], W)
	V = hcat(larmodel[1]...)
	chains = [convert(Lar.Cells, chain) for chain in larmodel[2:end]]
	return (V, chains...)
end

struct2lar (generic function with 1 method)

In [11]:
@btime struct2lar(structure)

  32.900 μs (142 allocations: 8.77 KiB)


([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4], [3, 1, 4, 2]])

In [12]:
@benchmark struct2lar(structure)

BenchmarkTools.Trial: 
  memory estimate:  8.77 KiB
  allocs estimate:  142
  --------------
  minimum time:     33.199 μs (0.00% GC)
  median time:      35.101 μs (0.00% GC)
  mean time:        50.967 μs (3.13% GC)
  maximum time:     20.217 ms (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [13]:
@code_warntype struct2lar(structure)

Variables
  #self#::Core.Compiler.Const(struct2lar, false)
  structure::LinearAlgebraicRepresentation.Struct
  @_3::Int64
  #10::var"#10#11"
  larmodel::Any
  W::Array{Array{Float64,1},1}
  V::Any
  chains::Any

Body::Tuple{Any,Vararg{Any,N} where N}
1 ─ %1  = Base.getproperty(Main.Lar, :evalStruct)::Any
│   %2  = (%1)(structure)::Any
│   %3  = Main.flatten(%2)::Tuple{Any,Array{Array{Float64,1},1}}
│   %4  = Base.indexed_iterate(%3, 1)::Core.Compiler.PartialStruct(Tuple{Any,Int64}, Any[Any, Core.Compiler.Const(2, false)])
│         (larmodel = Core.getfield(%4, 1))
│         (@_3 = Core.getfield(%4, 2))
│   %7  = Base.indexed_iterate(%3, 2, @_3::Core.Compiler.Const(2, false))::Core.Compiler.PartialStruct(Tuple{Array{Array{Float64,1},1},Int64}, Any[Array{Array{Float64,1},1}, Core.Compiler.Const(3, false)])
│         (W = Core.getfield(%7, 1))
│   %9  = Base.getindex(larmodel, 1)::Any
│         Main.append!(%9, W)
│   %11 = Base.getindex(larmodel, 1)::Any
│         (V = Core._apply_itera

## Modifiche

In [46]:
function flatten(listOfModels)
	W = Array{Float64,1}[]
	m = length(listOfModels[1])
	larmodel = [Array{Float64,1}[] for k=1:m]
	vertDict = Dict()
	index,defaultValue = 0,0
	for model in listOfModels
		V = model[1]
		for k=2:m
			for incell in model[k]
				outcell=[]
				for v in incell
					key = map(Lar.approxVal(7), V[:,v])
					if get(vertDict,key,defaultValue)==defaultValue
						index += 1
						vertDict[key]=index
						push!(outcell,index)
						push!(W,key)
					else
						push!(outcell,vertDict[key])
					end
				end
				append!(larmodel[k],[outcell])
			end
		end
	end
	return larmodel,W

end

flatten (generic function with 1 method)

In [47]:
function struct2lar(structure) 
	larmodel,W = flatten(Lar.evalStruct(structure))
	append!(larmodel[1], W)
	V = hcat(larmodel[1]...)
	chains = [convert(Lar.Cells, chain) for chain in larmodel[2:end]]
	return (V, chains...)
end

struct2lar (generic function with 1 method)

In [49]:
@btime struct2lar(structure)

  31.600 μs (127 allocations: 8.09 KiB)


([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4], [3, 1, 4, 2]])

In [17]:
@code_typed struct2lar(structure)

CodeInfo(
1 ─ %1  = Base.getproperty(Main.Lar, :evalStruct)::Any
│   %2  = (%1)(structure)::Any
│   %3  = Main.flatten(%2)::Tuple{Any,Array{Array{Float64,1},1}}
│   %4  = Base.getfield(%3, 1)::Any
│   %5  = Base.getfield(%3, 2)::Array{Array{Float64,1},1}
│   %6  = Base.getindex(%4, 1)::Any
│   %7  = Main.append!::Core.Compiler.Const(append!, false)
│   %8  = (isa)(%6, BitArray{1})::Bool
└──       goto #3 if not %8
2 ─ %10 = π (%6, BitArray{1})
│         invoke %7(%10::BitArray{1}, %5::Array{Array{Float64,1},1})::Any
└──       goto #4
3 ─       Main.append!(%6, %5)::Any
└──       goto #4
4 ┄ %15 = Base.getindex(%4, 1)::Any
│   %16 = Core._apply_iterate(Base.iterate, Main.hcat, %15)::Any
│   %17 = %new(Main.:(var"#16#17"))::Core.Compiler.Const(var"#16#17"(), false)
│   %18 = Base.lastindex(%4)::Any
│   %19 = (isa)(%18, Int64)::Bool
└──       goto #6 if not %19
5 ─ %21 = π (%18, Int64)
│   %22 = Base.sle_int(2, %21)::Bool
│   %23 = Base.sub_int(2, 1)::Int64
│   %24 = Base.ifelse(%22, %21,

In [18]:
@code_warntype struct2lar(structure)

Variables
  #self#::Core.Compiler.Const(struct2lar, false)
  structure::LinearAlgebraicRepresentation.Struct
  @_3::Int64
  #16::var"#16#17"
  larmodel::Any
  W::Array{Array{Float64,1},1}
  V::Any
  chains::Any

Body::Tuple{Any,Vararg{Any,N} where N}
1 ─ %1  = Base.getproperty(Main.Lar, :evalStruct)::Any
│   %2  = (%1)(structure)::Any
│   %3  = Main.flatten(%2)::Tuple{Any,Array{Array{Float64,1},1}}
│   %4  = Base.indexed_iterate(%3, 1)::Core.Compiler.PartialStruct(Tuple{Any,Int64}, Any[Any, Core.Compiler.Const(2, false)])
│         (larmodel = Core.getfield(%4, 1))
│         (@_3 = Core.getfield(%4, 2))
│   %7  = Base.indexed_iterate(%3, 2, @_3::Core.Compiler.Const(2, false))::Core.Compiler.PartialStruct(Tuple{Array{Array{Float64,1},1},Int64}, Any[Array{Array{Float64,1},1}, Core.Compiler.Const(3, false)])
│         (W = Core.getfield(%7, 1))
│   %9  = Base.getindex(larmodel, 1)::Any
│         Main.append!(%9, W)
│   %11 = Base.getindex(larmodel, 1)::Any
│         (V = Core._apply_itera